<a href="https://colab.research.google.com/github/Gwilson337/Project-4/blob/main/FinalProject/NFL_Fantasy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from pathlib import Path
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.ensemble import RandomForestRegressor
import random
import numpy as np

In [2]:
#Create file paths
csv_path1 = Path("/content/FantasyPros_2019.csv")
csv_path2 = Path("/content/FantasyPros_2020.csv")
csv_path3 = Path("/content/FantasyPros_2021.csv")
csv_path4 = Path("/content/FantasyPros_2022.csv")
csv_path5 = Path("/content/FantasyPros_2023.csv")

# Create DataFrames
df1 = pd.read_csv(csv_path1)
df2 = pd.read_csv(csv_path2)
df3 = pd.read_csv(csv_path3)
df4 = pd.read_csv(csv_path4)
df5 = pd.read_csv(csv_path5)

# #Join DataFrames
# all_df = pd.concat([df1, df2, df3, df4, df5], axis = 1)
# all_df.head()
# # all_df.shape

In [3]:
df1 = df1[["#", "Player", "Pos", "AVG", "TTL"]].rename(columns={"#" : "2019_Rank", "AVG" : "2019_AVG", "TTL" : "2019_TTL"})
df2 = df2[["#", "Player", "AVG", "TTL"]].rename(columns={"#" : "2020_Rank", "AVG" : "2020_AVG", "TTL" : "2020_TTL"})
df3 = df3[["#", "Player", "AVG", "TTL"]].rename(columns={"#" : "2021_Rank", "AVG" : "2021_AVG", "TTL" : "2021_TTL"})
df4 = df4[["#", "Player", "AVG", "TTL"]].rename(columns={"#" : "2022_Rank", "AVG" : "2022_AVG", "TTL" : "2022_TTL"})
df5 = df5[["Player", "TTL"]].rename(columns={"TTL" : "2023_TTL"})

all_df = pd.merge(df1, df2, on="Player", how="inner")
all_df = pd.merge(all_df, df3, on="Player", how="inner")
all_df = pd.merge(all_df, df4, on="Player", how="inner")
all_df = pd.merge(all_df, df5, on="Player", how="inner")

all_df

,2019_Rank,Player,Pos,2019_AVG,2019_TTL,2020_Rank,2020_AVG,2020_TTL,2021_Rank,2021_AVG,2021_TTL,2022_Rank,2022_AVG,2022_TTL,2023_TTL
0,1,Lamar Jackson,QB,28.1,421.7,10,22.8,341.8,19,21.1,253.0,19,20.3,243.1,338.2
1,2,Christian McCaffrey,RB,22.2,355.2,222,24.5,73.4,191,12.9,90.5,14,16.0,271.4,324.3
2,3,Dak Prescott,QB,21.8,348.8,91,27.7,138.6,8,20.7,330.6,28,17.8,213.6,351.8
3,4,Jameis Winston,QB,21.0,335.4,573,0.6,2.4,127,17.2,120.4,292,16.3,48.9,15.0
4,5,Russell Wilson,QB,20.9,333.6,6,23.3,372.8,20,17.8,248.8,22,15.7,235.8,264.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,565,Keith Kirkwood,WR,0.0,0.0,597,1.3,1.3,602,0.6,1.7,593,0.6,1.8,9.7
273,642,Tim Boyle,QB,0.0,-0.1,717,-0.1,-0.9,377,7.6,30.3,718,-0.9,-0.9,14.9
274,643,Nick Mullens,QB,-0.2,-0.3,109,12.8,128.3,498,9.9,9.9,468,3.2,12.8,74.7
275,648,Ray-Ray McCloud III,WR,-0.3,-2.0,466,0.8,12.2,388,1.7,27.2,311,2.6,44.1,16.5


In [4]:
# Check for missing values
# all_df.isnull().value_counts()

In [4]:
# Check data types
all_df.dtypes

2019_Rank      int64
Player        object
Pos           object
2019_AVG     float64
2019_TTL     float64
2020_Rank      int64
2020_AVG     float64
2020_TTL     float64
2021_Rank      int64
2021_AVG     float64
2021_TTL     float64
2022_Rank      int64
2022_AVG     float64
2022_TTL     float64
2023_TTL     float64
dtype: object

In [5]:
# Change rank column to float and verify
all_df[["2019_Rank", "2020_Rank", "2021_Rank", "2022_Rank"]] = all_df[["2019_Rank", "2020_Rank", "2021_Rank", "2022_Rank"]].astype("float")
all_df.dtypes

2019_Rank    float64
Player        object
Pos           object
2019_AVG     float64
2019_TTL     float64
2020_Rank    float64
2020_AVG     float64
2020_TTL     float64
2021_Rank    float64
2021_AVG     float64
2021_TTL     float64
2022_Rank    float64
2022_AVG     float64
2022_TTL     float64
2023_TTL     float64
dtype: object

In [6]:
# Set player name as index
all_df = all_df.set_index("Player")
all_df.head()

,2019_Rank,Pos,2019_AVG,2019_TTL,2020_Rank,2020_AVG,2020_TTL,2021_Rank,2021_AVG,2021_TTL,2022_Rank,2022_AVG,2022_TTL,2023_TTL
Player,,,,,,,,,,,,,,
Lamar Jackson,1.0,QB,28.1,421.7,10.0,22.8,341.8,19.0,21.1,253.0,19.0,20.3,243.1,338.2
Christian McCaffrey,2.0,RB,22.2,355.2,222.0,24.5,73.4,191.0,12.9,90.5,14.0,16.0,271.4,324.3
Dak Prescott,3.0,QB,21.8,348.8,91.0,27.7,138.6,8.0,20.7,330.6,28.0,17.8,213.6,351.8
Jameis Winston,4.0,QB,21.0,335.4,573.0,0.6,2.4,127.0,17.2,120.4,292.0,16.3,48.9,15.0
Russell Wilson,5.0,QB,20.9,333.6,6.0,23.3,372.8,20.0,17.8,248.8,22.0,15.7,235.8,264.9


In [7]:
# Get list of columns to store as features
all_df.columns

Index(['2019_Rank', 'Pos', '2019_AVG', '2019_TTL', '2020_Rank', '2020_AVG',
       '2020_TTL', '2021_Rank', '2021_AVG', '2021_TTL', '2022_Rank',
       '2022_AVG', '2022_TTL', '2023_TTL'],
      dtype='object')

In [8]:
# Drop position column and verify
all_df = all_df.drop(columns="Pos")
all_df.head()

,2019_Rank,2019_AVG,2019_TTL,2020_Rank,2020_AVG,2020_TTL,2021_Rank,2021_AVG,2021_TTL,2022_Rank,2022_AVG,2022_TTL,2023_TTL
Player,,,,,,,,,,,,,
Lamar Jackson,1.0,28.1,421.7,10.0,22.8,341.8,19.0,21.1,253.0,19.0,20.3,243.1,338.2
Christian McCaffrey,2.0,22.2,355.2,222.0,24.5,73.4,191.0,12.9,90.5,14.0,16.0,271.4,324.3
Dak Prescott,3.0,21.8,348.8,91.0,27.7,138.6,8.0,20.7,330.6,28.0,17.8,213.6,351.8
Jameis Winston,4.0,21.0,335.4,573.0,0.6,2.4,127.0,17.2,120.4,292.0,16.3,48.9,15.0
Russell Wilson,5.0,20.9,333.6,6.0,23.3,372.8,20.0,17.8,248.8,22.0,15.7,235.8,264.9


In [9]:
# Separate feautures from label
labelName = ["2023_TTL"]
featureNames = ['2019_Rank', '2019_AVG', '2019_TTL', '2020_Rank', '2020_AVG',
       '2020_TTL', '2021_Rank', '2021_AVG', '2021_TTL', '2022_Rank',
       '2022_AVG', '2022_TTL']
dfFeatures = all_df[featureNames]
dfFeatures.head()

,2019_Rank,2019_AVG,2019_TTL,2020_Rank,2020_AVG,2020_TTL,2021_Rank,2021_AVG,2021_TTL,2022_Rank,2022_AVG,2022_TTL
Player,,,,,,,,,,,,
Lamar Jackson,1.0,28.1,421.7,10.0,22.8,341.8,19.0,21.1,253.0,19.0,20.3,243.1
Christian McCaffrey,2.0,22.2,355.2,222.0,24.5,73.4,191.0,12.9,90.5,14.0,16.0,271.4
Dak Prescott,3.0,21.8,348.8,91.0,27.7,138.6,8.0,20.7,330.6,28.0,17.8,213.6
Jameis Winston,4.0,21.0,335.4,573.0,0.6,2.4,127.0,17.2,120.4,292.0,16.3,48.9
Russell Wilson,5.0,20.9,333.6,6.0,23.3,372.8,20.0,17.8,248.8,22.0,15.7,235.8


In [10]:
dfLabels = all_df[labelName]
dfLabels.head()

,2023_TTL
Player,
Lamar Jackson,338.2
Christian McCaffrey,324.3
Dak Prescott,351.8
Jameis Winston,15.0
Russell Wilson,264.9


In [11]:
labels = np.array(dfLabels)
features = np.array(dfFeatures)

In [12]:
train, test, trainLabels, testLabels = train_test_split(features, labels, test_size=(0.25), random_state=0)

In [13]:
rf = RandomForestRegressor(random_state=0)

In [14]:
rf.fit(train, trainLabels)

<ipython-input-14-faaa41696551>:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(train, trainLabels)


RandomForestRegressor(random_state=0)

In [15]:
rfPred = rf.predict(features)

In [16]:
# Create new dataframe, add columns, and check predictions vs actual data
fantasy_df = all_df[["2023_TTL"]]
fantasy_df["Points_Predicted"] = rfPred
fantasy_df["error"] = abs(fantasy_df["2023_TTL"]-fantasy_df["Points_Predicted"]).astype("int")

<ipython-input-16-0df004876128>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fantasy_df["Points_Predicted"] = rfPred
<ipython-input-16-0df004876128>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fantasy_df["error"] = abs(fantasy_df["2023_TTL"]-fantasy_df["Points_Predicted"]).astype("int")


In [17]:
# View model performance
fantasy_df.describe()

,2023_TTL,Points_Predicted,error
count,277.000000,277.000000,277.000000
mean,76.967509,73.908765,17.111913
std,76.356259,60.232628,23.966427
min,-1.300000,0.864000,0.000000
25%,14.900000,22.780000,4.000000
50%,53.700000,60.914000,10.000000
75%,128.000000,116.123000,21.000000
max,410.600000,325.946000,266.000000


In [18]:
# View prediction model dataframe
fantasy_df.head()

,2023_TTL,Points_Predicted,error
Player,,,
Lamar Jackson,338.2,325.946,12
Christian McCaffrey,324.3,310.102,14
Dak Prescott,351.8,318.160,33
Jameis Winston,15.0,75.136,60
Russell Wilson,264.9,268.145,3


In [19]:
# View in descending order
fantasy_df.sort_values("2023_TTL", ascending=False).head(20)

,2023_TTL,Points_Predicted,error
Player,,,
Josh Allen,410.6,144.387,266
Dak Prescott,351.8,318.160,33
Lamar Jackson,338.2,325.946,12
Christian McCaffrey,324.3,310.102,14
Jared Goff,303.1,182.293,120
Patrick Mahomes II,294.2,227.448,66
Baker Mayfield,284.1,198.300,85
Russell Wilson,264.9,268.145,3
Tyreek Hill,257.4,200.985,56
